In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import shap
import matplotlib.pyplot as plt
import numpy as np



In [2]:
df12 = pd.read_parquet('ThermocoupleData_2024-12_v2.parquet')
df1 = pd.read_parquet('ThermocoupleData_2025-01_v2.parquet')
df2 = pd.read_parquet('ThermocoupleData_2025-02_v2.parquet')
df3 = pd.read_parquet('ThermocoupleData_2025-03_v2.parquet')
df4 = pd.read_parquet('ThermocoupleData_2025-04_v2.parquet')

dfall = pd.concat([df12, df1, df2, df3, df4], ignore_index=True)




In [3]:
# count of all the df rows
print(f"df12: {len(df12)}")
print(f"df1: {len(df1)}")
print(f"df2: {len(df2)}")
print(f"df3: {len(df3)}")
print(f"df4: {len(df4)}")
print(f"dfall: {len(dfall)}")


df12: 44641
df1: 44641
df2: 40321
df3: 44581
df4: 33121
dfall: 207305


In [4]:
dfall.columns

Index(['DateTime', 'AvgTemp_ZON1_M', 'AvgTemp_ZON2_M', 'AvgTemp_ZON3_M',
       'AvgTemp_ZON4_M', 'AvgTemp_ZON5_M', 'AvgTemp_ZON6_M',
       'LineControlHastSverk4_1Act', 'UgnZon1BransleFlodeAr_Over',
       'UgnZon1BransleFlodeAr_Under', 'UgnZon1OljaFlodeAr_FT131',
       'UgnZon1TempRegAr_TC1', 'UgnZon1TempSkyddAr_TC2',
       'UgnZon1TempVaggOverBandAr_TC3', 'UgnZon1TempVaggUnderBandAr_TC4',
       'UgnZon2BransleFlodeAr_Over', 'UgnZon2BransleFlodeAr_Under',
       'UgnZon2OljaFlodeAr_FT231', 'UgnZon2TempAr_TC1',
       'UgnZon2TempSkyddAr_TC2', 'UgnZon2TempVaggOverBandAr_TC3',
       'UgnZon2TempVaggUnderBandAr_TC4', 'UgnZon3BransleFlodeAr_Over',
       'UgnZon3BransleFlodeAr_Under', 'UgnZon3OljaFlodeAr_FT331',
       'UgnZon3TempRegAr_TC1', 'UgnZon3TempSkyddAr_TC2',
       'UgnZon3TempVaggAr_TC3', 'UgnZon3Temp_TC4_Ar', 'UgnZon3Temp_TC5_Ar',
       'UgnZon4BransleFlodeAr_Over', 'UgnZon4BransleFlodeAr_Under',
       'UgnZon4OljaFlodeAr_FT431', 'UgnZon4TempAr_TC1',
       'UgnZon4Tem

In [5]:
dfzon1 = dfall[['AvgTemp_ZON1_M', 'DateTime', 'UgnZon1BransleFlodeAr_Under', 'UgnZon1BransleFlodeAr_Over',
                 'UgnZon1OljaFlodeAr_FT131', 'UgnZon1TempRegAr_TC1', 'UgnZon1TempSkyddAr_TC2',
       'UgnZon1TempVaggOverBandAr_TC3', 'UgnZon1TempVaggUnderBandAr_TC4', 'LineControlHastSverk4_1Act']]

In [ ]:
# Create lag features for dfzon1
# First, ensure the dataframe is sorted by DateTime
dfzon1_lag = dfzon1.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon1.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon1_lag[f"{col}_lag1"] = dfzon1_lag[col].shift(1)

# Reset index
dfzon1_lag = dfzon1_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon1 = dfzon1_lag.dropna().reset_index(drop=True)


Original shape: (207295, 19), New shape with lag features: (207305, 19)


In [14]:
dfzon2 = dfall[['AvgTemp_ZON2_M', 'DateTime', 'UgnZon2BransleFlodeAr_Under', 'UgnZon2BransleFlodeAr_Over',
                 'UgnZon2OljaFlodeAr_FT231', 'UgnZon2TempAr_TC1', 'UgnZon2TempSkyddAr_TC2',
         'UgnZon2TempVaggOverBandAr_TC3', 'UgnZon2TempVaggUnderBandAr_TC4', 'LineControlHastSverk4_1Act']]

In [15]:
# Create lag features for dfzon2
# First, ensure the dataframe is sorted by DateTime
dfzon2_lag = dfzon2.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon2.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon2_lag[f"{col}_lag1"] = dfzon2_lag[col].shift(1)

# Reset index
dfzon2_lag = dfzon2_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon2 = dfzon2_lag.dropna().reset_index(drop=True)


In [16]:
dfzon3 = dfall[['AvgTemp_ZON3_M', 'DateTime', 'UgnZon3BransleFlodeAr_Under', 'UgnZon3BransleFlodeAr_Over',
                    'UgnZon3OljaFlodeAr_FT331', 'UgnZon3TempRegAr_TC1', 'UgnZon3TempSkyddAr_TC2',
        'UgnZon3TempVaggAr_TC3', 'UgnZon3Temp_TC4_Ar', "UgnZon3Temp_TC5_Ar",  'LineControlHastSverk4_1Act']]

In [17]:
# Create lag features for dfzon3
# First, ensure the dataframe is sorted by DateTime
dfzon3_lag = dfzon3.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon3.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon3_lag[f"{col}_lag1"] = dfzon3_lag[col].shift(1)

# Reset index
dfzon3_lag = dfzon3_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon3 = dfzon3_lag.dropna().reset_index(drop=True)


In [18]:
dfzon4 = dfall[['AvgTemp_ZON4_M', 'DateTime', 'UgnZon4BransleFlodeAr_Under', 'UgnZon4BransleFlodeAr_Over',
                    'UgnZon4OljaFlodeAr_FT431', 'UgnZon4TempAr_TC1', 'UgnZon4TempSkyddAr_TC2',
        'UgnZon4TempVaggAr_TC3', 'UgnZon4TempVaggAr_TC4',  'LineControlHastSverk4_1Act']]

In [19]:
# Create lag features for dfzon4
# First, ensure the dataframe is sorted by DateTime
dfzon4_lag = dfzon4.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon4.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon4_lag[f"{col}_lag1"] = dfzon4_lag[col].shift(1)

# Reset index
dfzon4_lag = dfzon4_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon4 = dfzon4_lag.dropna().reset_index(drop=True)


In [20]:
dfzon5 = dfall[['AvgTemp_ZON5_M', 'DateTime',  
                    'UgnZon5OljaFlodeAr_FT531', 'UgnZon5TempAr_TC1', 'UgnZon5TempSkyddAr_TC2',
        'UgnZon5TempVaggAr_TC3', 'LineControlHastSverk4_1Act']]

In [21]:
# Create lag features for dfzon5
# First, ensure the dataframe is sorted by DateTime
dfzon5_lag = dfzon5.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon5.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon5_lag[f"{col}_lag1"] = dfzon5_lag[col].shift(1)

# Reset index
dfzon5_lag = dfzon5_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon5 = dfzon5_lag.dropna().reset_index(drop=True)


In [22]:
dfzon6 = dfall[['AvgTemp_ZON6_M', 'DateTime',
                    'UgnZon6OljaFlodeAr_FT631', 'UgnZon6TempAr_TC1', 'UgnZon6TempSkyddAr_TC2',
        'UgnZon6TempVaggAr_TC3', "UgnZon6TempUtgValvAr_TC5",  'LineControlHastSverk4_1Act']]

In [23]:
# Create lag features for dfzon6
# First, ensure the dataframe is sorted by DateTime
dfzon6_lag = dfzon6.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon6.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon6_lag[f"{col}_lag1"] = dfzon6_lag[col].shift(1)

# Reset index
dfzon6_lag = dfzon6_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon6 = dfzon6_lag.dropna().reset_index(drop=True)


In [24]:
# concatenate dfzon1 and dfzon2 together for furnace1 df
df_furnace1 = pd.concat([dfzon1, dfzon2], axis=1)
# Furnace 2 = Zones 3, 4, 5, and 6
df_furnace2 = pd.concat([dfzon3, dfzon4, dfzon5, dfzon6], axis=1)

In [29]:
df_furnace1 = df_furnace1.loc[:,~df_furnace1.columns.duplicated()]
df_furnace2 = df_furnace2.loc[:,~df_furnace2.columns.duplicated()]

In [30]:
df_furnace1.columns

Index(['AvgTemp_ZON1_M', 'DateTime', 'UgnZon1BransleFlodeAr_Under',
       'UgnZon1BransleFlodeAr_Over', 'UgnZon1OljaFlodeAr_FT131',
       'UgnZon1TempRegAr_TC1', 'UgnZon1TempSkyddAr_TC2',
       'UgnZon1TempVaggOverBandAr_TC3', 'UgnZon1TempVaggUnderBandAr_TC4',
       'LineControlHastSverk4_1Act', 'AvgTemp_ZON1_M_lag1',
       'UgnZon1BransleFlodeAr_Under_lag1', 'UgnZon1BransleFlodeAr_Over_lag1',
       'UgnZon1OljaFlodeAr_FT131_lag1', 'UgnZon1TempRegAr_TC1_lag1',
       'UgnZon1TempSkyddAr_TC2_lag1', 'UgnZon1TempVaggOverBandAr_TC3_lag1',
       'UgnZon1TempVaggUnderBandAr_TC4_lag1',
       'LineControlHastSverk4_1Act_lag1', 'AvgTemp_ZON2_M',
       'UgnZon2BransleFlodeAr_Under', 'UgnZon2BransleFlodeAr_Over',
       'UgnZon2OljaFlodeAr_FT231', 'UgnZon2TempAr_TC1',
       'UgnZon2TempSkyddAr_TC2', 'UgnZon2TempVaggOverBandAr_TC3',
       'UgnZon2TempVaggUnderBandAr_TC4', 'AvgTemp_ZON2_M_lag1',
       'UgnZon2BransleFlodeAr_Under_lag1', 'UgnZon2BransleFlodeAr_Over_lag1',
       'UgnZon2

In [31]:
df_furnace2.columns

Index(['AvgTemp_ZON3_M', 'DateTime', 'UgnZon3BransleFlodeAr_Under',
       'UgnZon3BransleFlodeAr_Over', 'UgnZon3OljaFlodeAr_FT331',
       'UgnZon3TempRegAr_TC1', 'UgnZon3TempSkyddAr_TC2',
       'UgnZon3TempVaggAr_TC3', 'UgnZon3Temp_TC4_Ar', 'UgnZon3Temp_TC5_Ar',
       'LineControlHastSverk4_1Act', 'AvgTemp_ZON3_M_lag1',
       'UgnZon3BransleFlodeAr_Under_lag1', 'UgnZon3BransleFlodeAr_Over_lag1',
       'UgnZon3OljaFlodeAr_FT331_lag1', 'UgnZon3TempRegAr_TC1_lag1',
       'UgnZon3TempSkyddAr_TC2_lag1', 'UgnZon3TempVaggAr_TC3_lag1',
       'UgnZon3Temp_TC4_Ar_lag1', 'UgnZon3Temp_TC5_Ar_lag1',
       'LineControlHastSverk4_1Act_lag1', 'AvgTemp_ZON4_M',
       'UgnZon4BransleFlodeAr_Under', 'UgnZon4BransleFlodeAr_Over',
       'UgnZon4OljaFlodeAr_FT431', 'UgnZon4TempAr_TC1',
       'UgnZon4TempSkyddAr_TC2', 'UgnZon4TempVaggAr_TC3',
       'UgnZon4TempVaggAr_TC4', 'AvgTemp_ZON4_M_lag1',
       'UgnZon4BransleFlodeAr_Under_lag1', 'UgnZon4BransleFlodeAr_Over_lag1',
       'UgnZon4OljaFlod

In [32]:
df_furnace2.shape

(207295, 59)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf # For seed setting if desired
import numpy as np
import joblib
import os

def train_autoencoder(df_zone, zone_name, model_dir="zone_models_final"):
    os.makedirs(model_dir, exist_ok=True)

    # Drop DateTime and rows with NaN
    df_model = df_zone.drop(columns=['DateTime']).dropna()

    if df_model.empty or len(df_model) < 2: 
        print(f"⚠️ Insufficient data for {zone_name} after dropping NaNs (rows: {len(df_model)}). Skipping training.")
        return

    
    test_s = 0.2
    if int(len(df_model) * test_s) == 0 and len(df_model) > 1: 
        if len(df_model) > 5: 
            test_s = 1 / len(df_model)
        else: 
            print(f"⚠️ Very few samples for {zone_name} ({len(df_model)}), cannot reliably split. Skipping.")
            return


    # shuffle=False is crucial for time-series data to prevent future data from leaking into the training set.
    X_train_df, X_test_df = train_test_split(df_model, test_size=test_s, shuffle=False)

    if X_train_df.empty or X_test_df.empty:
        print(f"⚠️ Not enough data for {zone_name} after train/test split (Train: {len(X_train_df)}, Test: {len(X_test_df)}). Skipping.")
        return

    # Normalize
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_df) # Fit ONLY on training data
    X_test = scaler.transform(X_test_df)     # Transform test data

    input_dim = X_train.shape[1]
    if input_dim == 0:
        print(f"⚠️ No features remaining for {zone_name} after scaling. Skipping.")
        return

    # Define simple autoencoder
    autoencoder = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(max(4, int(input_dim * 0.75)), activation='relu'), 
        layers.Dense(max(2, int(input_dim * 0.5)), activation='relu'),  
        layers.Dense(max(4, int(input_dim * 0.75)), activation='relu'),
        layers.Dense(input_dim, activation='linear')
    ])

    autoencoder.compile(optimizer='adam', loss='mse')

    # Train
    history = autoencoder.fit(
        X_train, X_train,
        epochs=50, 
        batch_size=32, 
        validation_data=(X_test, X_test),
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)], 
        verbose=0 
    )

    # Predict and calculate MSE
    X_test_pred = autoencoder.predict(X_test)
    mse = np.mean(np.square(X_test - X_test_pred), axis=1)

    if len(mse) == 0:
        print(f"⚠️ MSE calculation resulted in empty array for {zone_name}. Skipping threshold and saving.")
        return

    threshold = np.percentile(mse, 95) # Or 99, etc, as mentioned in the thesis report, this can be modified based on needs.

    # Save model, scaler, threshold
    model_path = os.path.join(model_dir, f"{zone_name}_autoencoder.keras")
    scaler_path = os.path.join(model_dir, f"{zone_name}_scaler.pkl")
    threshold_path = os.path.join(model_dir, f"{zone_name}_threshold.npy")

    try:
        autoencoder.save(model_path)
        joblib.dump(scaler, scaler_path)
        np.save(threshold_path, threshold)
        print(f"✅ Trained and saved model for {zone_name} | Threshold: {threshold:.4f} | Train Loss: {history.history['loss'][-1]:.4f} | Val Loss: {history.history['val_loss'][-1]:.4f}")
    except Exception as e:
        print(f"❌ Error saving artifacts for {zone_name}: {e}")


In [34]:
train_autoencoder(dfzon1, "zone1")
train_autoencoder(dfzon2, "zone2")
train_autoencoder(dfzon3, "zone3")
train_autoencoder(dfzon4, "zone4")
train_autoencoder(dfzon5, "zone5")
train_autoencoder(dfzon6, "zone6")


1296/1296 ━━━━━━━━━━━━━━━━━━━━ 1s 690us/step
✅ Trained and saved model for zone1 | Threshold: 0.0077 | Train Loss: 0.0016 | Val Loss: 0.0024
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 1s 650us/step
✅ Trained and saved model for zone2 | Threshold: 0.0099 | Train Loss: 0.0025 | Val Loss: 0.0034
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 1s 924us/step
✅ Trained and saved model for zone3 | Threshold: 0.0178 | Train Loss: 0.0012 | Val Loss: 0.0043
1227/1227 ━━━━━━━━━━━━━━━━━━━━ 1s 830us/step
✅ Trained and saved model for zone4 | Threshold: 0.0092 | Train Loss: 0.0008 | Val Loss: 0.0020
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 1s 685us/step
✅ Trained and saved model for zone5 | Threshold: 0.0333 | Train Loss: 0.0060 | Val Loss: 0.0101
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 1s 669us/step
✅ Trained and saved model for zone6 | Threshold: 0.0105 | Train Loss: 0.0023 | Val Loss: 0.0025


In [35]:
train_autoencoder(df_furnace1, "furnace1")
train_autoencoder(df_furnace2, "furnace2")

1296/1296 ━━━━━━━━━━━━━━━━━━━━ 1s 699us/step
✅ Trained and saved model for furnace1 | Threshold: 0.0127 | Train Loss: 0.0015 | Val Loss: 0.0032
1227/1227 ━━━━━━━━━━━━━━━━━━━━ 1s 673us/step
✅ Trained and saved model for furnace2 | Threshold: 0.0143 | Train Loss: 0.0013 | Val Loss: 0.0038


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import joblib
import os

def train_isolation_forest(df_zone, zone_name, model_dir="zone_models_final_isoforest"):
    
    os.makedirs(model_dir, exist_ok=True)

    # 1. Preprocessing: Drop DateTime and rows with NaN 
    df_model = df_zone.drop(columns=['DateTime']).dropna()

    if df_model.empty or len(df_model) < 2:
        print(f"⚠️ [IF] Insufficient data for {zone_name} after dropping NaNs (rows: {len(df_model)}). Skipping training.")
        return

    # 2. Train/Test Split: Crucially, shuffle=False for time-series data
    # Use the same test_size to ensure the split is comparable to the autoencoder's.
    X_train_df, X_test_df = train_test_split(df_model, test_size=0.2, shuffle=False)

    if X_train_df.empty or X_test_df.empty:
        print(f"⚠️ [IF] Not enough data for {zone_name} after train/test split. Skipping.")
        return

    # 3. Scaling: Fit on train, transform both train and test
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_df)
    X_test = scaler.transform(X_test_df)

    # 4. Model Definition and Training
    iso_forest = IsolationForest(contamination='auto', random_state=42, n_jobs=-1) # n_jobs=-1 uses all CPU cores
    
    iso_forest.fit(X_train)

    # 5. Calculate Anomaly Scores on the Test Set
    # A higher score will now mean MORE anomalous, just like reconstruction error.
    anomaly_scores = -1 * iso_forest.score_samples(X_test)

    if len(anomaly_scores) == 0:
        print(f"⚠️ [IF] Score calculation resulted in empty array for {zone_name}. Skipping.")
        return

    # 6. Determine the 95th Percentile Threshold
    # This threshold is same as AE threshold. again, this is just a placeholder and can be adjusted.
    threshold = np.percentile(anomaly_scores, 95)

    # 7. Save 
    model_path: str = os.path.join(model_dir, f"{zone_name}_isoforest.pkl")
    scaler_path = os.path.join(model_dir, f"{zone_name}_scaler_if.pkl") 
    threshold_path = os.path.join(model_dir, f"{zone_name}_threshold_if.npy")

    try:
        joblib.dump(iso_forest, model_path)
        joblib.dump(scaler, scaler_path)
        np.save(threshold_path, threshold)
        print(f"✅ [IF] Trained and saved model for {zone_name} | Threshold: {threshold:.4f}")
    except Exception as e:
        print(f"❌ [IF] Error saving artifacts for {zone_name}: {e}")

In [37]:
train_isolation_forest(dfzon1, "zone1")
train_isolation_forest(dfzon2, "zone2")
train_isolation_forest(dfzon3, "zone3")
train_isolation_forest(dfzon4, "zone4")
train_isolation_forest(dfzon5, "zone5")
train_isolation_forest(dfzon6, "zone6")
train_isolation_forest(df_furnace1, "furnace1")
train_isolation_forest(df_furnace2, "furnace2")

✅ [IF] Trained and saved model for zone1 | Threshold: 0.6172
✅ [IF] Trained and saved model for zone2 | Threshold: 0.6583
✅ [IF] Trained and saved model for zone3 | Threshold: 0.6491
✅ [IF] Trained and saved model for zone4 | Threshold: 0.6418
✅ [IF] Trained and saved model for zone5 | Threshold: 0.6987
✅ [IF] Trained and saved model for zone6 | Threshold: 0.6986
✅ [IF] Trained and saved model for furnace1 | Threshold: 0.6130
✅ [IF] Trained and saved model for furnace2 | Threshold: 0.6399


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
import joblib
import os

def train_thermocouple_regressor(df_zone_original, target_thermocouple_col, zone_name, model_type='ridge', model_dir="tc_regression_models_final_withlag"):
    
    os.makedirs(model_dir, exist_ok=True)
    print(f"\n--- Training regressor for {target_thermocouple_col} in {zone_name} (Model: {model_type}) ---")

    df_zone = df_zone_original.copy()

    # 1. Prepare Data
    if target_thermocouple_col not in df_zone.columns:
        print(f"❌ Target column '{target_thermocouple_col}' not found in DataFrame for {zone_name}. Skipping.")
        return

    # Drop DateTime and any rows with NaNs
    df_model = df_zone.drop(columns=['DateTime']).dropna(subset=[target_thermocouple_col]) 
    df_model = df_model.dropna() 

    if df_model.empty or len(df_model) < 2:
        print(f"⚠️ Insufficient data for {target_thermocouple_col} in {zone_name} after cleaning (rows: {len(df_model)}). Skipping.")
        return

    # Define features (X) and target (y)
    y = df_model[target_thermocouple_col]
    X = df_model.drop(columns=[target_thermocouple_col])

    if X.empty:
        print(f"⚠️ No features remaining for {target_thermocouple_col} in {zone_name}. Skipping.")
        return

    # 2. Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


    print(f"  ➤ Training samples: {X_train.shape[0]}")
    print(f"  ➤ Testing samples:  {X_test.shape[0]}")


    if X_train.empty or X_test.empty:
        print(f"⚠️ Not enough data for train/test split for {target_thermocouple_col} in {zone_name}. Skipping.")
        return

    # 3. Scale Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 4. Initialize and Train Model
    if model_type.lower() == 'ridge':
        model = Ridge(alpha=1.0)  # Alpha is the regularization strength, can be tuned
    elif model_type.lower() == 'lasso':
        model = Lasso(alpha=0.1) # Alpha for Lasso, can be tuned, lasso was not used in the final report!
    else:
        print(f"❌ Invalid model_type: {model_type}. Choose 'ridge' or 'lasso'. Skipping.")
        return

    try:
        model.fit(X_train_scaled, y_train)
    except Exception as e:
        print(f"❌ Error training model for {target_thermocouple_col} in {zone_name}: {e}. Skipping.")
        return

    # 5. Evaluate Model
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)

    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)

    print(f"Evaluation for {target_thermocouple_col}:")
    print(f"  Train MSE: {train_mse:.4f}, R2: {train_r2:.4f}")
    print(f"  Test MSE:  {test_mse:.4f}, R2: {test_r2:.4f}")

    # 6. Save Model and Scaler
    safe_target_name = target_thermocouple_col.replace('/', '_').replace('\\', '_')
    model_filename = f"{zone_name}_{safe_target_name}_{model_type}_regressor.pkl"
    scaler_filename = f"{zone_name}_{safe_target_name}_{model_type}_scaler.pkl"

    model_path = os.path.join(model_dir, model_filename)
    scaler_path = os.path.join(model_dir, scaler_filename)

    try:
        joblib.dump(model, model_path)
        joblib.dump(scaler, scaler_path)
        print(f"✅ Saved model to {model_path}")
        print(f"✅ Saved scaler to {scaler_path}")
    except Exception as e:
        print(f"❌ Error saving artifacts for {target_thermocouple_col} in {zone_name}: {e}")

    return model, scaler



In [40]:
# Define thermocouple columns per zone
thermocouple_cols_by_zone = {
    "ZON1": [
        'UgnZon1TempRegAr_TC1',
        'UgnZon1TempSkyddAr_TC2',
        'UgnZon1TempVaggOverBandAr_TC3',
        'UgnZon1TempVaggUnderBandAr_TC4'
    ],
    "ZON2": [
        'UgnZon2TempAr_TC1',
        'UgnZon2TempSkyddAr_TC2',
        'UgnZon2TempVaggOverBandAr_TC3',
        'UgnZon2TempVaggUnderBandAr_TC4'
    ],
    "ZON3": [
        'UgnZon3TempRegAr_TC1',
        'UgnZon3TempSkyddAr_TC2',
        'UgnZon3TempVaggAr_TC3',
        'UgnZon3Temp_TC4_Ar',
        'UgnZon3Temp_TC5_Ar'
    ],
    "ZON4": [
        'UgnZon4TempAr_TC1',
        'UgnZon4TempSkyddAr_TC2',
        'UgnZon4TempVaggAr_TC3',
        'UgnZon4TempVaggAr_TC4'
    ],
    "ZON5": [
        'UgnZon5TempAr_TC1',
        'UgnZon5TempSkyddAr_TC2',
        'UgnZon5TempVaggAr_TC3'
    ],
    "ZON6": [
        'UgnZon6TempAr_TC1',
        'UgnZon6TempSkyddAr_TC2',
        'UgnZon6TempVaggAr_TC3',
        'UgnZon6TempUtgValvAr_TC5'
    ]
}

# Define the dataframes per zone
df_by_zone = {
    "ZON1": dfzon1,
    "ZON2": dfzon2,
    "ZON3": dfzon3,
    "ZON4": dfzon4,
    "ZON5": dfzon5,
    "ZON6": dfzon6
}

# Train the regressors for each thermocouple in each zone
for zone, tc_cols in thermocouple_cols_by_zone.items():
    df = df_by_zone[zone]
    for tc_col in tc_cols:
        train_thermocouple_regressor(df, tc_col, zone, model_type='ridge')



--- Training regressor for UgnZon1TempRegAr_TC1 in ZON1 (Model: ridge) ---
  ➤ Training samples: 165836
  ➤ Testing samples:  41459
Evaluation for UgnZon1TempRegAr_TC1:
  Train MSE: 16.3865, R2: 0.9995
  Test MSE:  8.5935, R2: 0.9998
✅ Saved model to tc_regression_models_final_withlag\ZON1_UgnZon1TempRegAr_TC1_ridge_regressor.pkl
✅ Saved scaler to tc_regression_models_final_withlag\ZON1_UgnZon1TempRegAr_TC1_ridge_scaler.pkl

--- Training regressor for UgnZon1TempSkyddAr_TC2 in ZON1 (Model: ridge) ---
  ➤ Training samples: 165836
  ➤ Testing samples:  41459
Evaluation for UgnZon1TempSkyddAr_TC2:
  Train MSE: 12.6057, R2: 0.9996
  Test MSE:  7.3865, R2: 0.9999
✅ Saved model to tc_regression_models_final_withlag\ZON1_UgnZon1TempSkyddAr_TC2_ridge_regressor.pkl
✅ Saved scaler to tc_regression_models_final_withlag\ZON1_UgnZon1TempSkyddAr_TC2_ridge_scaler.pkl

--- Training regressor for UgnZon1TempVaggOverBandAr_TC3 in ZON1 (Model: ridge) ---
  ➤ Training samples: 165836
  ➤ Testing samples: